## 模型构造

### 继承modle类

定义$MLP$类重载$Module$类的$\_\_init\_\_$与$forward$函数，他们分别用于创建模型参数和定义前向计算。

In [2]:
import torch
from torch import nn
class MLP(nn.Module):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

实例化$MLP$函数得到变量$net$后，使用$net(X)$即可调用$Module$类的$\_\_call\_\_$函数，这个函数将调用$MLP$类定义的$forward$函数完成前向计算。

In [3]:
X = torch.rand(2,784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.2145, -0.0775,  0.0738,  0.1916,  0.3522, -0.0556, -0.0986,  0.1301,
         -0.2874, -0.0338],
        [-0.2162, -0.0454,  0.0949,  0.1890,  0.3329, -0.0278, -0.1261,  0.0561,
         -0.3367, -0.1008]], grad_fn=<AddmmBackward>)

### Module的子类


**Sequential类**

当模型的前向计算为简单串联各个层的计算时，$Sequential$类可以通过更加简单的方式定义模型。这正是$Sequential$类的目的：它可以接收一个子模块的有序字典$（OrderedDict）$或者一系列子模块作为参数来逐一添加$Module$的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

In [6]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict):
            for key, module in args[0].items():
                self.add_module(key, module)
        else:# 传入的不是OrderedDict
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
        
    def forward(self, input):
        for module in self._modules.values():
            input = module(input)
        return input

In [7]:
net = MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1637,  0.1670,  0.1474, -0.2663,  0.1106,  0.0148,  0.1949, -0.1489,
         -0.0070, -0.3519],
        [ 0.2637,  0.1461,  0.2253, -0.0681,  0.0542,  0.0903,  0.1135, -0.0210,
          0.0789, -0.2857]], grad_fn=<AddmmBackward>)

**ModuleList类**

$ModuleList$接收一个子模块的列表作为输入，然后也可以类似$List$那样进行$append$和$extend$操作

In [8]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


**ModuleList与Sequential类的比较**

$ModuleList$类只是一个存储模块的容器并没有实现$forward$功能，可以随意存放模块不用输入输出匹配。而$Sequential$类是一个实现了$forward$的容器，需要输入输出匹配。

$ModuleList$的出现只是让网络定义前向传播时更加灵活，见下面官网的例子。

In [15]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,10) for i in range(10)])
        
    def forward(self, input):
        for i, l in enumerate(self.linears):
#             print(i, l)
            input = self.linears[i//2](input)+l(input)
        return input

In [16]:
X = torch.rand(3, 10)
net = MyModule()
print(net)
net(X)

MyModule(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): Linear(in_features=10, out_features=10, bias=True)
  )
)


tensor([[-0.5144, -0.3573, -0.0951,  0.3442, -0.5037,  0.3322, -0.6466,  0.5391,
          0.1680, -0.7791],
        [-0.4765, -0.3672,  0.0277,  0.2228, -0.4427,  0.3426, -0.5691,  0.6005,
          0.1452, -0.7408],
        [-0.5021, -0.3504, -0.0580,  0.3128, -0.4784,  0.3392, -0.6160,  0.5642,
          0.1653, -0.7693]], grad_fn=<AddBackward0>)

另外，$ModuleList$不同于一般的$Python$的$list$，加入到$ModuleList$里面的所有模块的参数会被**自动添加到整个网络**中，下面看一个例子对比一下。

In [18]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])

class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        self.linears = [nn.Linear(10, 10)]

net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
print(net1)
for p in net1.parameters():
    print(p.size())

print("net2:")
print(net2)
for p in net2.parameters():
    print(p)

net1:
Module_ModuleList(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
  )
)
torch.Size([10, 10])
torch.Size([10])
net2:
Module_List()


### ModuleDict
$ModuleDict$接收一个子模块的字典作为输入, 然后也可以类似字典那样进行添加访问操作

In [19]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


和$ModuleList$一样，$ModuleDict$实例仅仅是存放了一些模块的字典，并没有定义$forward$函数需要自己定义。同样，$ModuleDict$也与$Python$的$Dict$有所不同，$ModuleDict$里的所有模块的参数会被自动添加到整个网络中。

### 构造复杂的模型
虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义forward函数，但直接继承Module类可以极大地**拓展模型构造的灵活性**。下面我们构造一个稍微复杂点的网络FancyMLP。在这个网络中，我们通过**get_constant函数创建训练中不被迭代的参数**，即**常数参数**。在前向计算中，除了使用创建的常数参数外，我们还使用**Tensor的函数和Python的控制流**，并**多次调用相同的层**。

In [20]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        
        self.rand_weight = torch.rand((20,20), requires_grad = False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, x):
        x = self.linear(x)
        # 使用constant参数
        x= nn.functional.relu(torch.mm(x, self.rand_weight.data)+1)
        # 复用
        x = self.linear(x)
        # 控制流，返回item进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

In [22]:
X = torch.rand(2, 20)

net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-0.9448, grad_fn=<SumBackward0>)

因为$FancyMLP$和$Sequential$类都是$Module$类的子类，所以我们可以嵌套调用它们。

In [23]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40,30), nn.ReLU())
        
    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-2.0518, grad_fn=<SumBackward0>)

+ 可以通过继承$Module$类来构造模型。
+ $Sequential$、$ModuleList$、$ModuleDict$类都继承自$Module$类。
+ 与$Sequential$不同，$ModuleList$和$ModuleDict$并没有定义一个完整的网络，它们只是将不同的模块存放在一起，需要自己定义$forward$函数。
+ 虽然$Sequential$等类可以使模型构造更加简单，但直接继承$Module$类可以极大地拓展模型构造的灵活性。